<a href="https://colab.research.google.com/github/Rahee93/DataEngineer_Zoomcamp_Workshop/blob/main/Ingestionwith_dlt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install dlt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 884.3/884.3 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 13.1 MB/s eta 0:00:00


In [58]:
#Question 1. The version (1 point)
!pip show dlt


Name: dlt
Version: 1.6.1
Summary: dlt is an open-source python-first scalable data loading library that does not require any backend to run.
Home-page: https://github.com/dlt-hub
Author: dltHub Inc.
Author-email: services@dlthub.com
License: Apache-2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: click, fsspec, gitpython, giturlparse, hexbytes, humanize, jsonpath-ng, makefun, orjson, packaging, pathvalidate, pendulum, pluggy, pytz, PyYAML, requests, requirements-parser, rich-argparse, semver, setuptools, simplejson, tenacity, tomlkit, typing-extensions, tzdata
Required-by: 


In [33]:
import dlt
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.paginators import PageNumberPaginator


In [37]:
from dlt.sources.helpers.rest_client import RESTClient
import dlt

@dlt.resource
def ny_taxi_data():
    api_url = "https://us-central1-dlthub-analytics.cloudfunctions.net/data_engineering_zoomcamp_api/ny_taxi_data"
    client = RESTClient(base_url=api_url)

    page_num = 1
    while True:
        params = {"page": page_num}
        response = client.get("/", params=params)

        if response.status_code == 200:
            data = response.json()
            if data:  # Check if data is not empty
                yield data
                page_num += 1
            else:
                break  # Break the loop if no more data
        else:
            print(f"Error fetching data: Status code {response.status_code}")
            break  # Stop on error

# The rest of your pipeline code...

In [38]:
# Create the pipeline
pipeline = dlt.pipeline(
    pipeline_name="ny_taxi_pipeline",
    destination="duckdb",
    dataset_name="ny_taxi_data"
)


In [39]:
load_info = pipeline.run(ny_taxi_data)  # Corrected function name
print(load_info)

Pipeline ny_taxi_pipeline load step completed in 2.08 seconds
1 load package(s) were loaded to destination duckdb and into dataset ny_taxi_data
The duckdb destination used duckdb:////content/ny_taxi_pipeline.duckdb location to store data
Load package 1739403738.9797974 is LOADED and contains no failed jobs


In [40]:
!pip install duckdb


In [42]:
#Question 2. How many tables were created? (1 point)
import duckdb
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [43]:
# Connect to the DuckDB database
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

In [44]:
# Create the schema if it doesn't exist
conn.sql(f"CREATE SCHEMA IF NOT EXISTS {pipeline.dataset_name}")

In [45]:
# Set search path to the dataset
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

In [46]:
# Describe the dataset
conn.sql("DESCRIBE").df()

,database,schema,name,column_names,column_types,temporary
0,ny_taxi_pipeline,ny_taxi_data,_dlt_loads,"[load_id, schema_name, status, inserted_at, sc...","[VARCHAR, VARCHAR, BIGINT, TIMESTAMP WITH TIME...",False
1,ny_taxi_pipeline,ny_taxi_data,_dlt_pipeline_state,"[version, engine_version, pipeline_name, state...","[BIGINT, BIGINT, VARCHAR, VARCHAR, TIMESTAMP W...",False
2,ny_taxi_pipeline,ny_taxi_data,_dlt_version,"[version, engine_version, inserted_at, schema_...","[BIGINT, BIGINT, TIMESTAMP WITH TIME ZONE, VAR...",False
3,ny_taxi_pipeline,ny_taxi_data,ny_taxi_data,"[end_lat, end_lon, fare_amt, passenger_count, ...","[DOUBLE, DOUBLE, DOUBLE, BIGINT, VARCHAR, DOUB...",False


In [51]:

conn.sql(f"SELECT * FROM information_schema.tables WHERE table_schema = '{pipeline.dataset_name}'").df()


,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action,TABLE_COMMENT
0,ny_taxi_pipeline,ny_taxi_data,ny_taxi_data,BASE TABLE,None,None,None,None,None,YES,NO,None,None
1,ny_taxi_pipeline,ny_taxi_data,_dlt_loads,BASE TABLE,None,None,None,None,None,YES,NO,None,None
2,ny_taxi_pipeline,ny_taxi_data,_dlt_pipeline_state,BASE TABLE,None,None,None,None,None,YES,NO,None,None
3,ny_taxi_pipeline,ny_taxi_data,_dlt_version,BASE TABLE,None,None,None,None,None,YES,NO,None,None


In [52]:
df = conn.sql("SELECT * FROM ny_taxi_data.ny_taxi_data LIMIT 10").df()
df


,end_lat,end_lon,fare_amt,passenger_count,payment_type,start_lat,start_lon,tip_amt,tolls_amt,total_amt,trip_distance,trip_dropoff_date_time,trip_pickup_date_time,surcharge,vendor_name,_dlt_load_id,_dlt_id,store_and_forward
0,40.742963,-73.980072,45.0,1,Credit,40.641525,-73.787442,9.0,4.15,58.15,17.52,2009-06-14 23:48:00+00:00,2009-06-14 23:23:00+00:00,0.0,VTS,1739403738.9797974,Oi4ALEL5+QicXQ,NaN
1,40.740187,-74.005698,6.5,1,Credit,40.722065,-74.009767,1.0,0.00,8.50,1.56,2009-06-18 17:43:00+00:00,2009-06-18 17:35:00+00:00,1.0,VTS,1739403738.9797974,scOXdK2QSMOgtw,NaN
2,40.718043,-74.004745,12.5,5,Credit,40.761945,-73.983038,2.0,0.00,15.50,3.37,2009-06-10 18:27:00+00:00,2009-06-10 18:08:00+00:00,1.0,VTS,1739403738.9797974,xGh1BWxWVylizQ,NaN
3,40.739637,-73.985233,4.9,1,CASH,40.749802,-73.992247,0.0,0.00,5.40,1.11,2009-06-14 23:58:00+00:00,2009-06-14 23:54:00+00:00,0.5,VTS,1739403738.9797974,4Dfz44Be59VLsQ,NaN
4,40.730032,-73.852693,25.7,1,CASH,40.776825,-73.949233,0.0,4.15,29.85,11.09,2009-06-13 13:23:00+00:00,2009-06-13 13:01:00+00:00,0.0,VTS,1739403738.9797974,t4wDrVfuQ6PJMA,NaN
5,40.777537,-73.976860,7.3,2,Credit,40.790582,-73.953652,2.0,0.00,10.30,2.10,2009-06-10 19:52:00+00:00,2009-06-10 19:43:00+00:00,1.0,VTS,1739403738.9797974,yduIKUcaZDRG1A,NaN
6,40.770277,-73.962125,3.7,1,Credit,40.767147,-73.966408,1.0,0.00,5.20,0.40,2009-06-10 20:09:00+00:00,2009-06-10 20:06:00+00:00,0.5,VTS,1739403738.9797974,8YxbUyLUv/XBKA,NaN
7,40.774043,-73.951465,8.1,2,CASH,40.761750,-73.977773,0.0,0.00,8.60,2.24,2009-06-14 21:08:00+00:00,2009-06-14 20:57:00+00:00,0.5,VTS,1739403738.9797974,Hc2x++6nnoAC1g,NaN
8,40.777985,-73.943683,6.1,1,CASH,40.766355,-73.959832,0.0,0.00,6.10,1.48,2009-06-14 12:56:00+00:00,2009-06-14 12:49:00+00:00,0.0,VTS,1739403738.9797974,9FvMkcf5uFdDjQ,NaN
9,40.720052,-74.009823,8.9,1,CASH,40.751327,-73.987588,0.0,0.00,9.90,2.72,2009-06-10 18:13:00+00:00,2009-06-10 18:03:00+00:00,1.0,VTS,1739403738.9797974,2dqhCKTGVIgbIw,NaN


In [53]:
#Question 3. The total number of records extracted? (1 point)
record_count = conn.sql("SELECT COUNT(*) FROM ny_taxi_data.ny_taxi_data").df()
record_count


,count_star()
0,10000


In [57]:
#Question 4. Average trip duration (1 point)
with pipeline.sql_client() as client:
    res = client.execute_sql(
        """
        SELECT
        AVG(date_diff('minute', trip_pickup_date_time, trip_dropoff_date_time))
        FROM ny_taxi_data.ny_taxi_data;
        """
    )
    # Prints column values of the first row
    print(res)


[(12.3049,)]
